# Retriever And Chain With Langchain

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader('F:/21_DAYS_NEW/PROJECTS/RAG_QA_Chatbot/data/attention.pdf')

docs = loader.load()
docs[:5]

f:\21_DAYS_NEW\PROJECTS\RAG_QA_Chatbot\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-08-03T00:07:29+00:00', 'author': '', 'keywords': '', 'moddate': '2023-08-03T00:07:29+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'F:/21_DAYS_NEW/PROJECTS/Advanced-RAG-QA-Chatbot-main/data/attention.pdf', 'total_pages': 15, 'page': 0, 'page_label': '1'}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗ †\nUniversity of Toronto\na

## Transform

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(docs)
documents[:5]

[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-08-03T00:07:29+00:00', 'author': '', 'keywords': '', 'moddate': '2023-08-03T00:07:29+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'F:/21_DAYS_NEW/PROJECTS/Advanced-RAG-QA-Chatbot-main/data/attention.pdf', 'total_pages': 15, 'page': 0, 'page_label': '1'}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗ †\nUniversity of Toronto\na

## Emdedd

In [3]:
# import the 0llamaEmbedding and FAISS vectorstore
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = HuggingFaceInstructEmbeddings()

db = FAISS.from_documents(documents[:30], embedding=embeddings)
db

C:\Users\jella\AppData\Local\Temp\ipykernel_36004\3705343406.py:5: LangChainDeprecationWarning: The class `HuggingFaceInstructEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceInstructEmbeddings()
C:\Users\jella\AppData\Local\Temp\ipykernel_36004\3705343406.py:5: LangChainDeprecationWarning: Default values for HuggingFaceInstructEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceInstructEmbeddings constructor instead.
  embeddings = HuggingFaceInstructEmbeddings()
No sentence-transformers model found with name hkunlp/instructor-large. Creating a new one with mean pooling.
Loading weights: 100%|██████████| 196/196 [00:00<00:0

In [4]:
query = 'An attention function can be described as mapping a query'
result = db.similarity_search(query)
result[0].page_content

`SentenceTransformer._target_device` has been deprecated, please use `SentenceTransformer.device` instead.


'3.2 Attention\nAn attention function can be described as mapping a query and a set of key-value pairs to an output,\nwhere the query, keys, values, and output are all vectors. The output is computed as a weighted sum\n3'

## Ollama LLM

In [5]:
# import GoogleGenerativeAI LLM
from langchain_google_genai import GoogleGenerativeAI
from getpass import getpass

api_key = getpass("Enter your Google API Key: ")

# Replace "models/text-bison-001" with a supported model
llm = GoogleGenerativeAI(model="gemini-2.5-flash", google_api_key=api_key)
llm


GoogleGenerativeAI(google_api_key=SecretStr('**********'), model='models/gemini-2.5-flash', client=ChatGoogleGenerativeAI(profile={'max_input_tokens': 1048576, 'max_output_tokens': 65536, 'image_inputs': True, 'audio_inputs': True, 'pdf_inputs': True, 'video_inputs': True, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'image_tool_message': True, 'tool_choice': True}, google_api_key=SecretStr('**********'), model='models/gemini-2.5-flash', client=<google.genai.client.Client object at 0x000001FF5A017230>, default_metadata=(), model_kwargs={}))

## Chat Prompt Template

In [6]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>
Question: {input}""")

## Chain

In [7]:
# Cahain introduction
# Create stuff Document Chain

from langchain_classic.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm, prompt)

- Retrievers: A retriever is an interface that returns documents given
 an unstructured query. It is more general than a vector store.
 A retriever does not need to be able to store documents, only to 
 return (or retrieve) them. Vector stores can be used as the backbone
 of a retriever, but there are other types of retrievers as well. 
 https://python.langchain.com/docs/modules/data_connection/retrievers/  

## Retriever

In [8]:
# set the retriever
retriever = db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceInstructEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001FE416FE510>, search_kwargs={})

- Retrieval chain:This chain takes in a user inquiry, which is then
passed to the retriever to fetch relevant documents. Those documents 
(and original inputs) are then passed to an LLM to generate a response
https://python.langchain.com/docs/modules/chains/

In [9]:
from langchain_classic.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [10]:
response = retrieval_chain.invoke({'input': 'Scaled Dot-Product Attention'})
response['answer']

`SentenceTransformer._target_device` has been deprecated, please use `SentenceTransformer.device` instead.


'Based on the provided context, here is a detailed description of Scaled Dot-Product Attention:\n\nScaled Dot-Product Attention is a specific type of attention mechanism (as illustrated in Figure 2, left). It operates by mapping a query and a set of key-value pairs to an output vector.\n\n**Inputs:**\nThe input to Scaled Dot-Product Attention consists of:\n*   Queries (Q) and keys (K) of dimension `dk`.\n*   Values (V) of dimension `dv`.\n\n**Computation Process:**\n1.  **Dot Products:** The dot products of the query with all keys are computed.\n2.  **Scaling:** Each of these dot products is then divided by the square root of `dk` (`√dk`).\n3.  **Softmax Function:** A softmax function is applied to the scaled dot products to obtain the weights assigned to each value.\n4.  **Weighted Sum:** The final output is computed as a weighted sum of the values, where the weights are derived from the compatibility function (scaled dot product followed by softmax) between the query and the correspo